In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C, Datasets as D

In [49]:
ids = [{'a' : 1},{'a' : 2, 'b' : 2},{'a' : 3, 'b' : 3, 'c' : 3}]

In [50]:
ids

[{'a': 1}, {'a': 2, 'b': 2}, {'a': 3, 'b': 3, 'c': 3}]

In [55]:
if {'a' : 3, 'b' : 3} in ids : print('si')

# Tutorial for the Dataset module

Dataset is the class used to build, perform and post-process a set made of several calculation performed both with QuantumESPRESSO and Yambo.

Here we discuss some explicit examples to describe the usage and the main features of the package.

## Perform a convergence analysis for the gs energy of Silicon

We use this class to find the value of the energy cutoff that guarantees a converged result for the
ground state energy of Silicon.

We start from a given input file for Silicon

In [3]:
inp = I.PwInput(file='IO_files/si_scf.in')
inp

{'control': {'verbosity': "'high'",
  'pseudo_dir': "'../pseudos'",
  'calculation': "'scf'",
  'prefix': "'si_scf'"},
 'system': {'force_symmorphic': '.true.',
  'occupations': "'fixed'",
  'ibrav': '2',
  'celldm(1)': '10.3',
  'ntyp': '1',
  'nat': '2',
  'ecutwfc': '40'},
 'electrons': {'conv_thr': '1e-08'},
 'ions': {},
 'cell': {},
 'atomic_species': {'Si': ['28.086', 'Si.pbe-mt_fhi.UPF']},
 'atomic_positions': {'type': 'crystal',
  'values': [['Si', [0.125, 0.125, 0.125]], ['Si', [-0.125, -0.125, -0.125]]]},
 'kpoints': {'type': 'automatic',
  'values': ([4.0, 4.0, 4.0], [0.0, 0.0, 0.0])},
 'cell_parameters': {},
 'file': 'IO_files/si_scf.in'}

And we define a Calculator that will be used by the Dataset class to run the computation

In [22]:
code = C.QeCalculator(mpi_run='mpirun -np 2', skip = True)
code.global_options()

Initialize a parallel QuantumESPRESSO calculator with scheduler direct


{'omp': 1,
 'executable': 'pw.x',
 'multiTask': True,
 'scheduler': 'direct',
 'mpi_run': 'mpirun -np 2',
 'cpus_per_task': 4,
 'ntasks': 3,
 'skip': True,
 'verbose': True}

Now we can define the instance of Dataset to perform the convergence procedure. Some information of the class
can be read as

In [23]:
D.Dataset?

Init signature: D.Dataset(label='Dataset', run_dir='runs', **kwargs)
Docstring:     
Class to perform a set of calculations and to manage the associated results.

Attributes:
    label (:py:class:`str`): the label of the dataset, it can be useful for instance if more
    than one istance of the class is present
    run_dir (:py:class:`str`): path of the directory where the runs will be performed
    **kwargs : all the parameters passed to the dataset and stored in its _global_options.
    Can be useful, for instance, in performing a post-processing of the results.

Example:
    >>> code = QeCalculator()
    >>> study=Dataset(label = .., run_dir = ..., **kwargs)
    >>> study.append_run(id={'ecut': 30, 'kpoints' : 4},input=...,runner=code)
    >>> study.append_run(id={'ecut': 40, 'kpoints' : 4},input=...,runner=code)
    >>> study.run()
Init docstring: Set the dataset ready for appending new runs
File:           ~/Applications/MPPI/mppi/Datasets/Dataset.py
Type:           type


In [31]:
gs_convergence = D.Dataset(label='Si_gs_convergence',run_dir='Si_gs_convergence', spin_orbit = False)

Dataset inherit from Runner so it has the same structure and we can use the same methods of QeCalculator and YamboCalculator 
to access to its global options. 

Note that in this case we have defined a spin_orbit variable that can be used later. This variables is 
stored in the global options of the dataset

In [32]:
gs_convergence.global_options()

{'label': 'Si_gs_convergence',
 'run_dir': 'Si_gs_convergence',
 'spin_orbit': False}

The next step is to append to the Dataset all the calculation that we want to peform lately.

For instance we can perform first a set of calculations in function of the cutoff energy

In [33]:
energy_cutoffs = [20,30,40,50] # in Ry

In [34]:
for e in energy_cutoffs:
    idd = {'eng_cut' : e} #id that identifies the run in the Dataset
    inp.set_prefix(D.name_from_id(idd)) #attribute the id as the prefix of the input
    inp.set_energy_cutoff(e)
    gs_convergence.append_run(id=idd,runner=code,input=inp, other_variables = e)

The append_run method set the attribute of the object, for instance

In [35]:
print(gs_convergence.ids) # identify each element of the dataset
print(gs_convergence.names) # name of the input file written on disk
print(gs_convergence.calculators)

[{'eng_cut': 20}, {'eng_cut': 30}, {'eng_cut': 40}, {'eng_cut': 50}]
['eng_cut_20', 'eng_cut_30', 'eng_cut_40', 'eng_cut_50']
[{'calc': <mppi.Calculators.QeCalculator.QeCalculator object at 0x7fdadd51fcf8>, 'runs': [0, 1, 2, 3]}]


The name of the input files is evaluated from the ids using the function name_from_id.

gs_convergence.runs is a list that contains the merge of the input object and the global options for each of the
appended run, in this way one can check which is the input associated.
Other variables passed in the append run are stored in the runs attribute. 

In [38]:
gs_convergence.runs[2] #give the parameters of the third computation appended to the dataset

{'label': 'Si_gs_convergence',
 'run_dir': 'Si_gs_convergence',
 'spin_orbit': False,
 'input': {'control': {'verbosity': "'high'",
   'pseudo_dir': "'../pseudos'",
   'calculation': "'scf'",
   'prefix': "'eng_cut_40'"},
  'system': {'force_symmorphic': '.true.',
   'occupations': "'fixed'",
   'ibrav': '2',
   'celldm(1)': '10.3',
   'ntyp': '1',
   'nat': '2',
   'ecutwfc': 40},
  'electrons': {'conv_thr': '1e-08'},
  'ions': {},
  'cell': {},
  'atomic_species': {'Si': ['28.086', 'Si.pbe-mt_fhi.UPF']},
  'atomic_positions': {'type': 'crystal',
   'values': [['Si', [0.125, 0.125, 0.125]],
    ['Si', [-0.125, -0.125, -0.125]]]},
  'kpoints': {'type': 'automatic',
   'values': ([4.0, 4.0, 4.0], [0.0, 0.0, 0.0])},
  'cell_parameters': {},
  'file': 'IO_files/si_scf.in'},
 'other_variables': 40}

The attribute .calculator is a dictionary that is empty before the run

In [15]:
gs_convergence.results

{}

Before run the Dataset we can add another computation, made with a different calculator

In [16]:
code2 = C.QeCalculator(omp = 2, mpi_run='mpirun -np 2',skip=True,verbose=True)

Initialize a parallel QuantumESPRESSO calculator with scheduler direct


In [17]:
# inp = I.PwInput() #activate this line to produce an empty input that produces a CRASH in the simulation
idd = 'code_number2' # we can use a string as id of the run
inp.set_prefix(D.name_from_id(idd))
inp.set_energy_cutoff(60)
gs_convergence.append_run(id=idd,runner=code2,input=inp) 

In [18]:
print(gs_convergence.ids)
gs_convergence.calculators

[{'eng_cut': 20}, {'eng_cut': 30}, {'eng_cut': 40}, {'eng_cut': 50}, 'code_number2']


[{'calc': <mppi.Calculators.QeCalculator.QeCalculator at 0x7fdadd57cf60>,
  'runs': [0, 1, 2, 3]},
 {'calc': <mppi.Calculators.QeCalculator.QeCalculator at 0x7fdadd51f1d0>,
  'runs': [4]}]

In [20]:
gs_convergence.runs[4]

{'label': 'Si_gs_convergence',
 'run_dir': 'Si_gs_convergence',
 'spin_orbit': False,
 'input': {'control': {'verbosity': "'high'",
   'pseudo_dir': "'../pseudos'",
   'calculation': "'scf'",
   'prefix': "'code_number2'"},
  'system': {'force_symmorphic': '.true.',
   'occupations': "'fixed'",
   'ibrav': '2',
   'celldm(1)': '10.3',
   'ntyp': '1',
   'nat': '2',
   'ecutwfc': 60},
  'electrons': {'conv_thr': '1e-08'},
  'ions': {},
  'cell': {},
  'atomic_species': {'Si': ['28.086', 'Si.pbe-mt_fhi.UPF']},
  'atomic_positions': {'type': 'crystal',
   'values': [['Si', [0.125, 0.125, 0.125]],
    ['Si', [-0.125, -0.125, -0.125]]]},
  'kpoints': {'type': 'automatic',
   'values': ([4.0, 4.0, 4.0], [0.0, 0.0, 0.0])},
  'cell_parameters': {},
  'file': 'IO_files/si_scf.in'}}

Once that all the computation have been added we can run the Dataset

In [65]:
gs_convergence.run()

Run directory Si_gs_convergence
Skip the computation for input eng_cut_20
Run directory Si_gs_convergence
Skip the computation for input eng_cut_30
Run directory Si_gs_convergence
Skip the computation for input eng_cut_40
Run directory Si_gs_convergence
Skip the computation for input eng_cut_50
Run directory Si_gs_convergence
Skip the computation for input code_number2


{0: 'Si_gs_convergence/eng_cut_20.save/data-file-schema.xml',
 1: 'Si_gs_convergence/eng_cut_30.save/data-file-schema.xml',
 2: 'Si_gs_convergence/eng_cut_40.save/data-file-schema.xml',
 3: 'Si_gs_convergence/eng_cut_50.save/data-file-schema.xml',
 4: 'Si_gs_convergence/code_number2.save/data-file-schema.xml'}

The run method returns the attribute .results of the Dataset. 

In [66]:
gs_convergence.results

{0: 'Si_gs_convergence/eng_cut_20.save/data-file-schema.xml',
 1: 'Si_gs_convergence/eng_cut_30.save/data-file-schema.xml',
 2: 'Si_gs_convergence/eng_cut_40.save/data-file-schema.xml',
 3: 'Si_gs_convergence/eng_cut_50.save/data-file-schema.xml',
 4: 'Si_gs_convergence/code_number2.save/data-file-schema.xml'}

This implementation allows us to parse the data after the execution of the dataset and/or to choose a parser 
among several choices. 

### Parsing of the results

One way to perform the parsing of the results is _a posteriori_ from the run of the dataset.

For instance we can parse the results with the PwParser class of this package

In [67]:
from mppi import Parsers as P

results = {}
for run,data in gs_convergence.results.items():
    results[run] = P.PwParser(data)

Parse file : Si_gs_convergence/eng_cut_20.save/data-file-schema.xml
Parse file : Si_gs_convergence/eng_cut_30.save/data-file-schema.xml
Parse file : Si_gs_convergence/eng_cut_40.save/data-file-schema.xml
Parse file : Si_gs_convergence/eng_cut_50.save/data-file-schema.xml
Parse file : Si_gs_convergence/code_number2.save/data-file-schema.xml


In [68]:
results

{0: <mppi.Parsers.PwParser.PwParser at 0x7f4dfd5fd518>,
 1: <mppi.Parsers.PwParser.PwParser at 0x7f4dfd5bb588>,
 2: <mppi.Parsers.PwParser.PwParser at 0x7f4dfd5bb438>,
 3: <mppi.Parsers.PwParser.PwParser at 0x7f4dfd56fb70>,
 4: <mppi.Parsers.PwParser.PwParser at 0x7f4dfd526f28>}

The results associate to the key "i" correspond to the i-th element appended to the run.

The input parameters associated to each key of results are written inside the gs_convergence_runs[key] list.

For instance the total energy is extracted as

In [69]:
for run,res in results.items():
    print('run',run,'energy',res.get_energy(convert_eV=False))

run 0 energy -7.870821313426413
run 1 energy -7.872953197509275
run 2 energy -7.874327291306248
run 3 energy -7.874492376334014
run 4 energy -7.874513952356973


### Usage of the post processing function

The Parsing, or other more specific procedures, can be performed directly when the run method is called.

To do so, we define a post processing function and pass it to the Dataset. 

The class will apply this function when the run method of Dataset is called. For istance in this way we can directly 
extract the total energy 

In [70]:
def extract_energy(dataset): 
    from mppi import Parsers as P
    energy = {}
    for run,data in dataset.results.items():
        results = P.PwParser(data,verbose=False)
        energy[run] = results.get_energy(convert_eV = False)
    return energy

In [71]:
gs_convergence.set_postprocessing_function(extract_energy)

Once that the post processing function is passed to dataset it is directly applied when the run is executed

In [72]:
code.update_global_options(verbose=False)
code2.update_global_options(verbose=False)
gs_convergence.run()

{0: -7.870821313426413,
 1: -7.872953197509275,
 2: -7.874327291306248,
 3: -7.874492376334014,
 4: -7.874513952356973}

Note that the attribute results contains always the name of the xml data, the post processed results
can be accessed in the class as self.post_processing(). 

For this reason the method fetch_results has been slightly modified with respect to the original implementation
of PyBigDFT.

### Usage of the fetch_results method

Another possible approach is to define a post processing function that perform a simple parsing of the data.

Then we can use fetch_results to seek for the attribute energy in the computation(s) that match the id 
passed in fetch_results

In [74]:
def parse_data(dataset):
    from mppi import Parsers as P
    results = {}
    for run,data in dataset.results.items():
        results[run] = P.PwParser(data,verbose=False)
    return results

In [75]:
gs_convergence.set_postprocessing_function(parse_data)

In [76]:
gs_convergence.run()

{0: <mppi.Parsers.PwParser.PwParser at 0x7f4dfd562630>,
 1: <mppi.Parsers.PwParser.PwParser at 0x7f4dfd7a7cc0>,
 2: <mppi.Parsers.PwParser.PwParser at 0x7f4dfd4d2320>,
 3: <mppi.Parsers.PwParser.PwParser at 0x7f4dfd4a0630>,
 4: <mppi.Parsers.PwParser.PwParser at 0x7f4dfd44aba8>}

In [77]:
gs_convergence.fetch_results(id={'eng_cut': 50},attribute='energy')

[-7.874492376334014]

### Usage of the seek_convergence method

We present the functionality of this method by performing a second convergence test on the number of kpoints.

In this example we set the energy cutoff to 60 Ry and build a new dataset appending run with increasing number of
kpoints.

In [78]:
inp = I.PwInput('IO_files/si_scf.in')
inp.set_energy_cutoff(60)

In [80]:
code = C.QeCalculator(skip=True,verbose=False)
#code.global_options()

Initialize a QuantumESPRESSO calculator with OMP_NUM_THREADS=1 and command mpirun -np 4 pw.x


In [81]:
gs_kpoint = D.Dataset(label='Si_kpoints_convergence',run_dir='Si_gs_convergence')

In [82]:
kpoints = [2,3,4,5,6,7,8]

In [84]:
for k in kpoints:
    id = {'kp':k}
    inp.set_kpoints(points = [k,k,k])
    inp.set_prefix(D.name_from_id(id))
    gs_kpoint.append_run(id=id,runner=code,input=inp)

The runs have been appended but not performed, then we call seek_convergence.

We want to perform a convergence procedure based on the value of the total energy of the system.
So we can use the post processing function that directly provides this quantity

In [85]:
gs_kpoint.set_postprocessing_function(extract_energy)

In [86]:
gs_kpoint.seek_convergence(rtol=0.001)

Fetching results for id " {'kp': 2} "
Fetching results for id " {'kp': 3} "
Fetching results for id " {'kp': 4} "
Fetching results for id " {'kp': 5} "
Convergence reached in Dataset "Si_kpoints_convergence" for id " {'kp': 4} "


({'kp': 4}, -7.874513952262473)

Seek_converge runs all the computation (in the order provided by append_run) until convergence is reached.
Otherwise it is possible to pass a list of ids as argument of the method, in this case the calculation are restricted
to the simulations associated to the provided ids.

It is also possible to use a more generic post processing function that simply parse the data.
In this case we can choose which quantity is used to check if the convergence is reached by specifying the attribute = ...
options in the call of the seek_convergence. For instance

In [87]:
gs_kpoint.set_postprocessing_function(parse_data)

In [98]:
gs_kpoint.seek_convergence(rtol=0.001,attribute='energy')

Fetching results for id " {'kp': 2} "
Fetching results for id " {'kp': 3} "
Fetching results for id " {'kp': 4} "
Fetching results for id " {'kp': 5} "
Convergence reached in Dataset "Si_kpoints_convergence" for id " {'kp': 4} "


({'kp': 4}, -7.874513952262473)

## Perform a convergence test for Hartree-Fock computations with Yambo

We consider a set of Hartree-Fock computation for silicon and we look for the value of the EXXRLvcs that ensure
a converged value of the direct gap.

First of all we need a nscf computation. We start from scf result with ecutoff = 60 and kpoints = [4,4,4]

In [3]:
inp = I.PwInput('Si_gs_convergence/kp_4.in')
inp.set_nscf(8)
inp.set_kpoints(points = [6,6,6]) #nscf kpoints can be different from the scf
name = 'nscf_kp6_ecut60'
inp.set_prefix(name)
#inp

In [117]:
code = C.QeCalculator()
code.global_options()

Initialize a QuantumESPRESSO calculator with OMP_NUM_THREADS=1 and command mpirun -np 4 pw.x


{'omp': 1,
 'mpi_run': 'mpirun -np 4',
 'executable': 'pw.x',
 'skip': False,
 'verbose': True}

In [118]:
code.run(run_dir='Si_gs_convergence',input=inp,name=name,source_dir='Si_gs_convergence/kp_4.save')

delete log file: Si_gs_convergence/nscf_kp6_ecut60.log
delete xml file: Si_gs_convergence/nscf_kp6_ecut60.xml
delete folder: Si_gs_convergence/nscf_kp6_ecut60.save
Copy source_dir Si_gs_convergence/kp_4.save in the Si_gs_convergence/nscf_kp6_ecut60.save
Run directory Si_gs_convergence
Executing command: mpirun -np 4 pw.x -inp nscf_kp6_ecut60.in > nscf_kp6_ecut60.log


'Si_gs_convergence/nscf_kp6_ecut60.save/data-file-schema.xml'

The next step is the generation of the run_dir and SAVE folder

In [119]:
from mppi import Utilities as U

In [3]:
run_dir = 'Si_hf_convergence'
source_dir = 'Si_gs_convergence/nscf_kp6_ecut60.save'

In [122]:
U.build_SAVE(source_dir,run_dir)

Create folder Si_hf_convergence
Executing command: cd Si_gs_convergence/nscf_kp6_ecut60.save; p2y -a 2
Executing command: cp -r Si_gs_convergence/nscf_kp6_ecut60.save/SAVE Si_hf_convergence
Executing command: cd Si_hf_convergence;OMP_NUM_THREADS=1 yambo


Now we are ready to build the Yambo dataset

In [4]:
code = C.YamboCalculator(skip=True)

Initialize a Yambo calculator with OMP_NUM_THREADS=1 and command mpirun -np 4 yambo


In [5]:
inp = I.YamboInput(args='yambo -x -V rl',folder=run_dir)
# we are interested at the direct gap at Gamma so we include only the first kpoint
inp['variables']['QPkrange'] = [[1,1,1,8],'']
inp

{'args': 'yambo -x -V rl',
 'folder': 'Si_hf_convergence',
 'filename': 'yambo.in',
 'arguments': ['HF_and_locXC'],
 'variables': {'FFTGvecs': [2733.0, 'RL'],
  'SE_Threads': [0.0, ''],
  'EXXRLvcs': [17153.0, 'RL'],
  'QPkrange': [[1, 1, 1, 8], '']}}

In [6]:
hf_convergence = D.Dataset(label='Si_hf',run_dir=run_dir)

Let us start by adding some computations to see how to manage the data

In [7]:
exx_values = [1.,2.,3.,4.] #in Hartree

In [8]:
for e in exx_values:
    id = {'exxrl' : e}
    inp['variables']['EXXRLvcs'] = [1e3*e, 'mHa']
    hf_convergence.append_run(id=id,input=inp,runner=code)

 If needed we can also pass the jobname attribute by adding, for istance
 
 jobname=D.name_from_id(id)+'-job' 
 
 in the appen_run

In [9]:
hf_convergence.run()

Run directory Si_hf_convergence
Skip the computation for input exxrl_1.0
Run directory Si_hf_convergence
Skip the computation for input exxrl_2.0
Run directory Si_hf_convergence
Skip the computation for input exxrl_3.0
Run directory Si_hf_convergence
Skip the computation for input exxrl_4.0


{0: {'output': ['Si_hf_convergence/exxrl_1.0/o-exxrl_1.0.hf'],
  'ndb': ['Si_hf_convergence/exxrl_1.0/ndb.HF_and_locXC']},
 1: {'output': ['Si_hf_convergence/exxrl_2.0/o-exxrl_2.0.hf'],
  'ndb': ['Si_hf_convergence/exxrl_2.0/ndb.HF_and_locXC']},
 2: {'output': ['Si_hf_convergence/exxrl_3.0/o-exxrl_3.0.hf'],
  'ndb': ['Si_hf_convergence/exxrl_3.0/ndb.HF_and_locXC']},
 3: {'output': ['Si_hf_convergence/exxrl_4.0/o-exxrl_4.0.hf'],
  'ndb': ['Si_hf_convergence/exxrl_4.0/ndb.HF_and_locXC']}}

### Parsing the results with a post processing function

We can define a general post processing function to extract all the results from the o- files of the dataset.

We can use the YamboParser class of this package

In [10]:
def parse_data(dataset):
    from mppi import Parsers as P
    results = {}
    for run,data in dataset.results.items():
        results[run] = P.YamboParser(data['output'],verbose=True)
    return results

In [11]:
hf_convergence.set_postprocessing_function(parse_data)

In [12]:
code.update_global_options(verbose=False,skip=True)
results = hf_convergence.run()

Parse file Si_hf_convergence/exxrl_1.0/o-exxrl_1.0.hf
Parse file Si_hf_convergence/exxrl_2.0/o-exxrl_2.0.hf
Parse file Si_hf_convergence/exxrl_3.0/o-exxrl_3.0.hf
Parse file Si_hf_convergence/exxrl_4.0/o-exxrl_4.0.hf


Results can be extracted as

In [14]:
for irun in results:
    print(results[irun]['hf']['ehf'])

[-19.05416   -1.101     -1.067     -1.62535    6.79846    6.790734
   6.649487   7.654567]
[-19.13128   -1.552     -1.519     -2.08017    6.501243   6.493162
   6.351671   7.156224]
[-19.16567   -1.701     -1.667     -2.22218    6.34695    6.329222
   6.187717   6.996501]
[-19.17205   -1.718     -1.684     -2.23912    6.328658   6.310536
   6.169065   6.977456]


### Computing the direct gap with a post processing function

We describe the usage of a post processing function to perform a more specific operation like computing
the direct band gap. We define the post processing function

In [59]:
def get_direct_gap(dataset):
    """"
    Compute the direct band gap assuming that there is only one kpoint.
    The arguments energy_col, val_band and cond_band are read from the global_options
    of the dataset.
    """
    from mppi import Parsers as P
    import numpy as np
    glob_opt = dataset.global_options()
    val_band = glob_opt.get('val_band',1)
    cond_band = glob_opt.get('cond_band',1)
    # the name of the column used to compute the gap
    energy_col = glob_opt.get('energy_col','hf') 
    gap = {}
    for run,data in dataset.results.items():
        results = P.YamboParser(data['output'])
        key = list(results.keys())[0] # select the key (can be hf or qp)
        bands = results[key]['band']
        index_val = np.where(bands == val_band)
        index_cond = np.where(bands == cond_band)
        energy = results[key][energy_col]
        delta = energy[index_cond]-energy[index_val]
        gap[run] = float(delta)
    return gap

This function assume that some input like the specification of the conduction and valence bands are given in the global options
of the dataset. So we can se

In [60]:
hf_convergence.update_global_options(val_band = 4, cond_band = 5, energy_col = 'hf')

Then we set the new post processing function and run the dataset

In [61]:
hf_convergence.set_postprocessing_function(get_direct_gap)

In [62]:
hf_convergence.run()

{0: 6.5168870000000005, 1: 6.67449, 2: 6.662207, 3: 6.660855000000001}

### Usage of seek convergence

The post processing function defined above can be used together with the seek_convergence method to perform a convergence study

In this case we define a new dataset and append many possible runs. Only those one needed to reach the given tolerance will be executed

In [63]:
hf_convergence2 = D.Dataset(label='Si_hf',run_dir=run_dir,val_band = 4, cond_band = 5, var_name = 'hf')

In [64]:
exx_values = [float(i) for i in range(1,10)] #in Hartree
exx_values

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [65]:
for e in exx_values:
    id = {'exxrl' : e}
    inp['variables']['EXXRLvcs'] = [1e3*e, 'mHa']
    hf_convergence2.append_run(id=id,input=inp,runner=code)

In [66]:
hf_convergence2.set_postprocessing_function(get_direct_gap)

In [67]:
hf_convergence2.seek_convergence(rtol=0.0001)

Fetching results for id " {'exxrl': 1.0} "
Fetching results for id " {'exxrl': 2.0} "
Fetching results for id " {'exxrl': 3.0} "
Fetching results for id " {'exxrl': 4.0} "
Fetching results for id " {'exxrl': 5.0} "
Fetching results for id " {'exxrl': 6.0} "
Convergence reached in Dataset "Si_hf" for id " {'exxrl': 5.0} "


({'exxrl': 5.0}, 6.661784)